### 实现思路(ID3)
    1. 自定义信息熵计算函数，用于计算数据集的信息熵
    2. 自定义数据划分函数，用于根据指定特征的指定取值，划分数据集
    3. step2的自数据集作为输入给step1的函数，可以计算出按某指定特征的某指定取值(A=ai)划分的数据集的信息熵H(Di)，同时计算按某指定特征的某指定取值(A=ai)划分的数据集的样本概率|Di|/|D|
    4. 遍历该特征各个取值，计算各取值下划分的数据集的信息熵H(Di)和样本概率|Di|/|D|，相乘，再求和得到得到特征A对数据集D的经验条件熵H(D|A)
    5. 计算特征A对数据集的信息增益g(D,A)=H(D)-H(D|A)
    6. 以此类推，计算各特征对数据集的信息增益，取信息增益最大的特征为最佳划分特征，得到树T1
    7. 对T1各结点继续step3-6,选择信息增益最大的特征，继续划分数据，得到新的决策树
    8. 直到信息增益小于阈值，或无特征可划分，或每个分支下的所有实例都具有相同的分类，决策树完成
- **注意，ID3一直在分支，容易过拟合，因此需要对决策树剪枝，提高对测试集数据预测的性能**

In [1]:
import pandas as pd
import numpy as np
import operator
from math import log


"""
信息熵计算函数，用于计算数据集的信息熵
输入：数据集，每一行是一条数据，最后一列是各条数据集的类别
输出：该数据集的信息熵
思路：
建立一个字典，对数据集各数据的类别计数，
从而计算各类别出现频率(作为概率pi)，
最后调用信息熵公式计算 H(D)=-求和(pi*logpi)
"""
def calEntropy(dataset):
    n=len(dataset)
    labelCounts={}
    
    #对数据集各数据的类别计数
    for data in dataset:
        datalabel=data[-1] #取data最后一列，类别列
        if datalabel not in labelCounts.keys():
            labelCounts[datalabel]=0
        labelCounts[datalabel]+=1
    
    entropy=0.0
    
    #计算各类别出现频率(作为概率pi),调用信息熵公式计算 H(D)=-求和(pi*logpi)
    for key in labelCounts.keys():
        prob=float(labelCounts[key])/n
        entropy -= prob*log(prob,2)
    return entropy


"""
数据划分函数，用于根据指定特征的指定取值，划分数据集
输入：数据集、特征所在列索引、特征取值
输出：满足指定特征等于指定取值的数据子集
"""
def splitDataset(dataset,index,value):
    subDataset=[]
    for data in dataset:
        if data[index]==value:
            #抽取除了data[index]的内容(一个特征用于计算其对数据集的经验条件熵时，不需要此特征在子数据集中)
            splitData=data[:index] #取索引之前的元素
            splitData.extend(data[index+1:]) #再合并索引之后的元素
            subDataset.append(splitData)
    return subDataset


"""
选择信息增益最大的特征作为数据集划分特征
输入：数据集
输出：该数据集的最佳划分特征
"""
def chooseFeature(dataset):
    #初始化
    numFeature=len(dataset[0])-1 #因为最后一列是类别
    baseEntropy=calEntropy(dataset) #H(D)
    bestInfoGain=0.0
    bestFeatureIndex=-1
    
    #创建特征A各取值a的列表
    for i in range(numFeature):
        featureList=[data[i] for data in dataset]
        uniqueValue=set(featureList)
        empEntropy=0.0 #初始化特征A对数据集D的经验条件熵H(D|A)
        
        #计算特征A各取值a的信息熵H(Di)和样本概率|Di|/|D|，并相乘
        for value in uniqueValue:
            subDataset=splitDataset(dataset,i,value) #(列索引为i的特征)特征A取value值所划分的子数据集
            prob=len(subDataset)/float(len(dataset)) #计算|Di|/|D|
            empEntropy += prob*calEntropy(subDataset) #H(D|A)
        
        #取信息增益最大的特征为最佳划分特征
        infoGain=baseEntropy-empEntropy #信息增益
        if infoGain>bestInfoGain:
            bestInfoGain=infoGain
            bestFeatureIndex=i
    return bestFeatureIndex


"""
对数据集各数据类别进行计数排序
"""
def majorClass(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote]=0
        classCount[vote]+=1
    
    #对classCount按value降序排序
    sortedClassCount=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0] #返回类别最大的类别名



"""
主函数：递归构建决策树
输入：数据集(list类型)，数据集特征列表(按在数据集的位置排序)(list类型)
输出：该数据集的决策树
思路：【递归】
    1. 若数据集属于同一类，则返回该类别，划分停止
    2. 若数据集所有特征已经遍历，返回当前计数最多的类别为该结点类别，划分停止
    3. 否则继续分支，调用chooseFeature()函数，选择当前数据集最优特征
    4. 遍历当前最优特征各属性值，划分数据集，并递归调用自身createTree()构建子数据集的决策树
    5. 完成
"""
def createTree(dataset,featureLabels):
    classList=[data[-1] for data in dataset] #取数据集各数据类别
    
    #若数据集属于同一类，则返回该类别，划分停止
    if classList.count(classList[0])==len(classList):
        return classList[0]
    
    #若数据集所有特征已经遍历，返回当前计数最多的类别为该结点类别，划分停止
    if len(dataset[0])==1:
        return majorClass(classList)
    
    #否则继续分支，调用chooseFeature()函数，选择当前数据集最优特征
    bestFeatureIndex=chooseFeature(dataset)
    bestFeature=featureLabels[bestFeatureIndex]
    
    #用于存储决策树，字典结构存储树的所有信息，并可体现包含关系
    desitionTree={bestFeature:{}} 
    del(featureLabels[bestFeatureIndex]) #删除已被用于划分数据的特征
    
    #得到当前最优划分特征的各属性值
    featureValues=[data[bestFeatureIndex] for data in dataset]
    uniqueValues=set(featureValues)
    
    #遍历当前最优特征各属性值，划分数据集，并递归调用自身createTree()构建子数据集的决策树
    for value in uniqueValues:
        #得到已删除当前最优划分特征的特征列表,用于递归调用
        subFeatureLabels=featureLabels[:] 
       
        #用当前最优划分特征的指定值分割子数据集，用于递归调用
        subData=splitDataset(dataset,bestFeatureIndex,value) 
        desitionTree[bestFeature][value]=createTree(subData,subFeatureLabels)
    return desitionTree

#### 测试
- 西瓜分类数据集测试

In [2]:
watermalon=pd.read_csv(r"D:\python\data\watermalon.txt",sep="\t")
watermalon.shape

(17, 7)

In [5]:
watermalon_list=np.array(watermalon).tolist() #构建数据集
watermalon_list

[['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '是'],
 ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '是'],
 ['乌黑', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '是'],
 ['青绿', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '是'],
 ['浅白', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '是'],
 ['青绿', '稍蜷', '浊响', '清晰', '稍凹', '软粘', '是'],
 ['乌黑', '稍蜷', '浊响', '稍糊', '稍凹', '软粘', '是'],
 ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', '是'],
 ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '硬滑', '否'],
 ['青绿', '硬挺', '清脆', '清晰', '平坦', '软粘', '否'],
 ['浅白', '硬挺', '清脆', '模糊', '平坦', '硬滑', '否'],
 ['浅白', '蜷缩', '浊响', '模糊', '平坦', '软粘', '否'],
 ['青绿', '稍蜷', '浊响', '稍糊', '凹陷', '硬滑', '否'],
 ['浅白', '稍蜷', '沉闷', '稍糊', '凹陷', '硬滑', '否'],
 ['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '软粘', '否'],
 ['浅白', '蜷缩', '浊响', '模糊', '平坦', '硬滑', '否'],
 ['青绿', '蜷缩', '沉闷', '稍糊', '稍凹', '硬滑', '否']]

In [6]:
features=watermalon.columns.tolist()[0:-1] #提取特征列表
my_tree=createTree(watermalon_list,features)
my_tree

{'纹理': {'模糊': '否',
  '清晰': {'根蒂': {'硬挺': '否',
    '稍蜷': {'色泽': {'乌黑': {'触感': {'硬滑': '是', '软粘': '否'}}, '青绿': '是'}},
    '蜷缩': '是'}},
  '稍糊': {'触感': {'硬滑': '否', '软粘': '是'}}}}